In [ ]:
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import numpy as np
import os

In [3]:
dataset_dir = 'F:/images'

datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input, 
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.4
)

train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    classes = ["elliptical", "spiral", "uncertain"],
    subset='training'
)

valid_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    classes=["elliptical", "spiral", "uncertain"],
    subset='validation'
)

test_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    classes=["elliptical", "spiral", "uncertain"],
    subset='validation'
)

Found 45636 images belonging to 3 classes.
Found 30423 images belonging to 3 classes.
Found 30423 images belonging to 3 classes.


In [4]:
x, y = test_generator.next()
x.shape

(32, 224, 224, 3)

In [6]:
train_generator.num_classes

3

In [7]:
base_model = ResNet50(include_top=False, weights='imagenet')
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False
    
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_generator, validation_data=valid_generator, epochs=10)

94765736/94765736 [==============================] - 60s 1us/step
Epoch 1/10
1427/1427 [==============================] - ETA: 0s - loss: 0.8664 - accuracy: 0.6361

KeyboardInterrupt: 